In [50]:
#Dependencies and Pre-processing
from rake_nltk import Rake
import pandas as pd
import requests
# from IPython.display import Image, display
from IPython.core.display import display, HTML
from timeit import default_timer as timer
import pprint
import sys
from timeit import default_timer as timer
import gensim
import string
from gensim.models import word2vec
import numpy as np
Songs = pd.read_csv("songdata.csv")
Songs.set_index("song",inplace=True)
Songs.drop(["artist","link"],inplace=True,axis=1)
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', limit=1000000, binary=True)


In [55]:
#Functions
def pixabay_get(keywords,N):
    r = requests.get('https://pixabay.com/api/?key=6713313-c85a6d8e3f6fe1a13b85f7d78&q='+"+".join(keywords)+'&image_type=photo&page=1&per_page='+str(N))
    result=[]
    try:
        for img in r.json()['hits']:
            result.append({'url':img['webformatURL'], 'tags':img["tags"].split(", ")})
    except:
        pass
    return result

def fragment(lyrics):
    paragraphs = []
    paragraphs = lyrics.split('\n  \n')
    if (len(paragraphs)==1):
        paragraphs = lyrics.split('  \r\n  ')
    return [{'text':x} for x in paragraphs]

def extract_kw(text):
    rake = Rake()
    rake.extract_keywords_from_text(text)
    A = rake.get_ranked_phrases_with_scores()
    if not A:
        return []
    return [x[1] for x in A]

def paragraph_to_keywords(paragraphs):
    for p in paragraphs:
        listofkeywords = extract_kw(p['text'])
        if (listofkeywords == []):
            paragraphs.remove(p)
            continue
        p['keywords'] = listofkeywords

def get_images_and_tags(paragraphs):
    for p in paragraphs:
        p['imgs']= []
        for kw in p['keywords']:
            for img in pixabay_get([kw],50):
                if (img): p['imgs'].append(img)
                    
def custom_display(result_para):
    imgs_html = ''
    lyrics_html = ''
    lyrics = ''
    for p in result_para:
        imgs_html += '<img src="'+p['picked_url']+'"></img>'
    for p in result_para:
        lyrics += p['text']
    for line in lyrics.split('\n'):
        kw_set = set()
        for p in result_para:
            for kw in p['keywords']:
                kw_set.add(kw)
        for kw in kw_set:
            line = line.replace(kw, '<span style="color:#42A5F5;">'+kw+'</span>')
        lyrics_html += '<div style="font-size:20px;">'+line+'</div>'
    display(HTML('<div style="display:flex"><div style="width:25%">'+imgs_html+'</div><div style="margin-left:2rem; width:75%;line-height:1.8;">'+lyrics_html+'</div></div>'))

In [52]:
def SongVisualize(index):
    #Choose a song
    lyrics = Songs.iloc[index]["text"]

    ################################################################

    #Fragment into paragraphs
    paragraphs = fragment(lyrics)

    ################################################################

    paragraph_to_keywords(paragraphs)

    ################################################################

    get_images_and_tags(paragraphs)

    ################################################################

    #Compare vectors and Pick the most relevent image for each paragraph

    # Calculate kw_avg vector for each image 
    for p in paragraphs:
        kw_avg=np.zeros(300)
        kw_count=0
        for kw in p['keywords']:
            for kw_nospace in kw.split(' '):
                try: 
                    kw_avg += model.wv[kw_nospace]
                    kw_count += 1
                except: # ignore if keyword is not found by the model
                    pass
        if (kw_count > 0):
            kw_avg /= kw_count
            p['kw_avg']=kw_avg

    # Calculate vector for each image
    for p in paragraphs:
        for img in p['imgs']:
            img_tag_avg = np.zeros(300)
            img_tag_count = 0
            for tag in img['tags']:
                img_tag_count += 1
            try:
                    img_tag_avg += model.wv[tag]
            except:
                    pass
            if (img_tag_count>0):
                img['vector'] = img_tag_avg / img_tag_count

    #Pick the most relevent image
    filtered = []
    for p in paragraphs:
        difference = np.inf
        picked_url = ''
        for img in p['imgs']:
            if (img['url'] in filtered):
                continue
            if (difference > np.sum(np.abs(p['kw_avg'] - img['vector']))):
                difference = np.sum(np.abs(p['kw_avg'] - img['vector']))
                picked_url = img['url']
        p['picked_url'] = picked_url 
        filtered.append(picked_url)

    ################################################################################
    
    return paragraphs

In [57]:
result = SongVisualize(100)

In [58]:
custom_display(result)